In [1]:
from get_students_info import get_student_info

In [3]:
get_student_info('data_files/stuscho.csv')

Default attributes are: "IBTEACH", "WEALTH", "ESCS", "SC013Q01TA", "SCIERES"
Default output name is: student_info.csv
Using default settings?(y/n): n
Input attribute name, "s" to stop:
invalid_name
No attribute: invalid_name
Please input again.
WEALTH
s
Input output file name:customized_student_info.csv
